# Fixed Portfolio Database Builder
This notebook recreates `portfolio.db` correctly.

In [12]:

import sqlite3
import yfinance as yf
import pandas as pd
from datetime import date, timedelta

DB_PATH = "portfolio.db"
TICKERS = ["AAPL","MSFT","AMZN","TSLA","JPM","JNJ","NVDA","SPY"]
END_DATE = date.today()
START_DATE = END_DATE - timedelta(days=365)

# Drop old tables
conn = sqlite3.connect(DB_PATH)
conn.execute("DROP TABLE IF EXISTS prices;")
conn.execute("DROP TABLE IF EXISTS portfolio_weights;")
conn.commit()
conn.close()


In [ ]:
raw = yf.download(
    TICKERS,
    start=START_DATE,
    end=END_DATE,
    auto_adjust=True,
    progress=False
)

# "Close" is now adjusted automatically 
data = raw["Close"]

df = (
    data.reset_index()
        .melt(id_vars="Date", var_name="ticker", value_name="adj_close")
        .dropna()
)



In [15]:

# Create correct schema
conn = sqlite3.connect(DB_PATH)
conn.execute("""
CREATE TABLE IF NOT EXISTS prices (
    Date TEXT,
    ticker TEXT,
    adj_close REAL
);
""")
conn.commit()
conn.close()


In [16]:
# Convert pandas Timestamp → string ("YYYY-MM-DD")
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

conn = sqlite3.connect(DB_PATH)
insert_query = "INSERT INTO prices (Date, ticker, adj_close) VALUES (?, ?, ?)"
conn.executemany(insert_query, df.values.tolist())
conn.commit()
conn.close()


In [17]:

# Create weights
weights = pd.DataFrame({
    "ticker": ["AAPL","MSFT","AMZN","TSLA","JPM","JNJ","NVDA"],
    "weight": [0.15,0.15,0.15,0.15,0.10,0.10,0.20]
})

conn = sqlite3.connect(DB_PATH)
conn.execute("CREATE TABLE IF NOT EXISTS portfolio_weights (ticker TEXT, weight REAL)")
conn.executemany("INSERT INTO portfolio_weights VALUES (?, ?)", weights.values.tolist())
conn.commit()
conn.close()

weights


,ticker,weight
0,AAPL,0.15
1,MSFT,0.15
2,AMZN,0.15
3,TSLA,0.15
4,JPM,0.10
5,JNJ,0.10
6,NVDA,0.20


In [18]:

# Verify schema
conn = sqlite3.connect(DB_PATH)
print(conn.execute("PRAGMA table_info(prices)").fetchall())
conn.close()


[(0, 'Date', 'TEXT', 0, None, 0), (1, 'ticker', 'TEXT', 0, None, 0), (2, 'adj_close', 'REAL', 0, None, 0)]


In [19]:
conn = sqlite3.connect(DB_PATH)
print(conn.execute("SELECT * FROM prices LIMIT 20").fetchall())
conn.close()


[('2024-11-22', 'AAPL', 228.8350372314453), ('2024-11-25', 'AAPL', 231.821533203125), ('2024-11-26', 'AAPL', 234.0016632080078), ('2024-11-27', 'AAPL', 233.87225341796875), ('2024-11-29', 'AAPL', 236.26145935058594), ('2024-12-02', 'AAPL', 238.51126098632812), ('2024-12-03', 'AAPL', 241.5574951171875), ('2024-12-04', 'AAPL', 241.91587829589844), ('2024-12-05', 'AAPL', 241.94573974609375), ('2024-12-06', 'AAPL', 241.7466278076172), ('2024-12-09', 'AAPL', 245.6390380859375), ('2024-12-10', 'AAPL', 246.65444946289062), ('2024-12-11', 'AAPL', 245.38018798828125), ('2024-12-12', 'AAPL', 246.8435821533203), ('2024-12-13', 'AAPL', 247.0128173828125), ('2024-12-16', 'AAPL', 249.90969848632812), ('2024-12-17', 'AAPL', 252.33872985839844), ('2024-12-18', 'AAPL', 246.9331817626953), ('2024-12-19', 'AAPL', 248.66534423828125), ('2024-12-20', 'AAPL', 253.34420776367188)]
